In [ ]:
# ✅ ngrok バイナリをダウンロード＆展開＆配置
!wget -q -c https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from huggingface_hub import login
from getpass import getpass

# 🤫 Hugging Face トークン
hf_token = getpass("🤫 Hugging Face access token を入力してください: ")
login(token=hf_token)

# 🔐 ngrok トークン
ngrok_token = getpass("🔐 ngrok token を入力してください: ")
!ngrok config add-authtoken {ngrok_token}

🤫 Hugging Face access token を入力してください: ··········
🔐 ngrok token を入力してください: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(MODEL, use_auth_token=hf_token)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype="auto",
#     use_auth_token=hf_token
# ).to("cuda")  # or device_map="auto", low_cpu_mem_usage=False

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype="auto",
    # device_map="auto",
    use_auth_token=hf_token
).to("cuda")

# 🧠 パイプライン作成（Flaskの上で定義しておく）
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
for name, param in model.named_parameters():
    print(name, param.device)
    break  # 1つだけ確認すればOK


model.embed_tokens.weight cuda:0


In [ ]:
import subprocess
import time
import requests

# 🔄 ngrok 起動（バックグラウンド）
subprocess.Popen(["ngrok", "http", "5000"])

# ⏳ ngrokのURL取得まで待機
time.sleep(7)

try:
    res = requests.get("http://localhost:4040/api/tunnels")
    tunnels = res.json()["tunnels"]
    tunnel_url = [t["public_url"] for t in tunnels if t["proto"] == "https"][0]
    print("🌍 Colab構造化API URL（.envに貼る用）:")
    print(tunnel_url)
except Exception as e:
    print("❌ ngrok URL取得エラー:", e)




from flask import Flask, request, jsonify
import json, threading
import sys
import regex as re

app = Flask(__name__)

def extract_json(text):
    try:
        # すべての { ... } を正規表現で探して、最後のだけ使う
        matches = list(re.finditer(r"\{(?:[^{}]|(?R))*\}", text, re.DOTALL))
        if matches:
            json_str = matches[-1].group()
            print("🔍 抽出したJSON候補:", json_str)
            return json.loads(json_str)
        return {}
    except Exception as e:
        print("❌ JSON抽出エラー:", e)
        return {}

@app.route("/ping", methods=["GET"])
def ping():
    return "pong"

def call_llm(natural_text: str):
    prompt = f"""
以下の自然文を読み取り、次の形式で予約情報をJSONとして出力してください。

特に「priority_score（優先度）」は、以下のルールを参考に0.0〜1.0の範囲で適切に判断してください：

【優先度の判断基準】
・締め切りが早い → 高く評価（例：「今日中」「明日まで」など）
・研究のインパクトが大きい（例：「論文」「国際学会」など）→ 高く評価
・外部資金があるプロジェクト（例：「共同研究」「科研費」など）→ 高く評価
・自己学習目的（例：「勉強のため」など）→ 優先度を下げる
・緊急性が高い場合（例：「感染症対策」など）→ 高く評価

【出力形式】
```json
{{
  "start_time": "2025-06-01T08:00:00",
  "end_time": "2025-06-01T10:00:00",
  "purpose": "共同研究のための学習実験",
  "priority_score": 0.85,
  "server_name": "A100",
  "received_text": "{natural_text}"
}}


自然文: {natural_text}
"""

    result = pipe(prompt, max_new_tokens=256, temperature=0.7)[0]["generated_text"]
    print("🧠 LLM出力:\n", result)

    json_result = extract_json(result)

    # 🔽 JSONをファイルに保存する！
    with open("/content/llm_output_log.json", "w", encoding="utf-8") as f:
        f.write(json.dumps({
            "input_text": natural_text,
            "raw_output": result,
            "parsed_json": json_result
        }, ensure_ascii=False))
        f.write("\n\n")  # 区切り用

    return json_result


@app.route("/analyze", methods=["POST"])
def analyze():
    data = request.get_json()
    natural_text = data.get("text", "")

    try:
        with open("/content/flask_log.txt", "w", encoding="utf-8") as f:
            f.write("🔥🔥🔥 /analyze が呼び出された！ 🔥🔥🔥\n")
            f.write(f"📥 受け取った自然文: {natural_text}\n\n")
    except Exception as e:
        print("❌ ログファイル書き込みエラー:", e)

    try:
        result = call_llm(natural_text)
        print("✅ 構造化結果:", result)

        return jsonify({
            "start_time": result.get("start_time"),
            "end_time": result.get("end_time"),
            "purpose": result.get("purpose"),
            "priority_score": result.get("priority_score"),
            "server_name": result.get("server_name"),
            "received_text": natural_text
        })
    except Exception as e:
        print("❌ LLM解析エラー:", e)
        return jsonify({"error": str(e)}), 500


# ✅ Flask をバックグラウンドで起動（Colabが止まらない）
def run_flask():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_flask).start()


🌍 Colab構造化API URL（.envに貼る用）:
https://a041-35-190-180-134.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
# ==============================
# 🔌 停止セル：ngrok と Flask を明示的に終了
# ==============================

import os
import signal

# ✅ ngrok のプロセス終了（プロセス名ベース）
!pkill ngrok

# ✅ Flask のスレッド（threading.Thread）を止めるには、通常はランタイム再起動が一番確実だが、
#     一応「サーバーを停止するルート」も用意しておけばリモート停止できる（ただしColabでは不安定）

import requests

try:
    # Flask に shutdown 要求を送る（アプリに /shutdown が定義されている場合）
    res = requests.post("http://localhost:5000/shutdown")
    print("🛑 Flask 停止応答:", res.text)
except Exception as e:
    print("⚠️ Flask 停止リクエスト失敗:", e)


INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 08:05:41] "POST /shutdown HTTP/1.1" 404 -


🛑 Flask 停止応答: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

